# 使用 LangChain 实现 AutoGPT

In [ ]:
%pip -q install  openai tiktoken
%pip install git+https://github.com/hwchase17/langchain

In [8]:
import os

os.environ["OPENAI_API_KEY"] = ""
# SERPAPI_API_KEY
os.environ["SERPAPI_API_KEY"] = ""

我们来设置一些工具，这包括搜索工具、写文件工具和读文件工具，这些都是为了让AutoGPT能够更好地完成任务。

In [10]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

设置内存，这是用于存储模型在执行任务过程中的中间步骤。这对于理解模型的工作过程和改进模型的性能都非常重要。

In [11]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

我们使用OpenAI 的嵌入模型, 向量库使用 FAISS。

In [12]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

在设置好工具和内存后，初始化模型和AutoGPT。我们选择的模型是ChatOpenAI，这是一个专为对话设计的大语言模型。

In [13]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

Langchain 已经封装好 AutoGPT 类，我们通过 from_llm_and_tools 方法可以实例化这个类。

In [15]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
)
# Set verbose to be true
agent.chain.verbose = True

接下来，运行一个示例，让AutoGPT为旧金山编写一份天气报告。这展示了AutoGPT在实际应用中的能力。

In [ ]:
agent.run(["上海最近都要下雨吗？"])


最后，设置聊天历史记录的内存。这是除了用于存储模型中间步骤的内存之外的另一种内存。这种内存默认使用'ChatMessageHistory'，但可以改变。当你希望使用不同类型的内存，例如'FileChatHistoryMemory'时，这将非常有用。

In [ ]:
from langchain.memory.chat_message_histories import FileChatMessageHistory

agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
    chat_history_memory=FileChatMessageHistory("chat_history.txt"),
)

In [ ]:
agent.run(["write a weather report for Shanghai today"])